### This is project to get the title and paragraph description from nasa.mars.gov website and upload the data in mongodb

In [106]:
import gmaps
import json
import requests

import random
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Access maps with unique API key

from flask import Flask, render_template
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time

from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import pymongo


#### Initializes the Mongo DB connection

In [107]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


   executable_path = {'executable_path': 'chromedriver.exe'}
    return Browser('chrome', **executable_path, headless=False)

#### This is the DB initialization and updating the data

#### Initializing the browser

In [108]:
executable_path = {'executable_path': 'chromedriver.exe'}
#return Browser('chrome', **executable_path, headless=False)
browser= Browser('chrome', **executable_path, headless=False)


##### This is the mars nasa gov url to get the title and paragraph

In [109]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

time.sleep(1)

# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")


avg_temps = soup.find("ul", class_="item_list")

#a_all = avg_temps.find_all('a')[1].text
#td_all = avg_temps.find_all('td')[3].text
vol_str_data = ''
int_vol_str = ''

avg_vol_str_data = ''
avg_int_vol_str = ''

mkt_vol_str_data = ''
mkt_int_vol_str = ''
title_list=[]
paragraphs = avg_temps.find_all('div', class_="list_text")
for para in paragraphs:
    data_symbol={}
    try:
        data_symbol["title"] = para.find('div', class_="content_title").get_text()
        data_symbol["paragraph"] = para.find('div', class_="article_teaser_body").get_text()
        title_list.append(data_symbol)
    except:
        pass
title_list


[{'title': 'Sensors on Mars 2020 Spacecraft Answer Long-Distance Call From Earth',
  'paragraph': "Instruments tailored to collect data during the descent of NASA's next rover through the Red Planet's atmosphere have been checked in flight."},
 {'title': "NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars",
  'paragraph': "For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design."},
 {'title': "NASA InSight's 'Mole' Is Out of Sight",
  'paragraph': "Now that the heat probe is just below the Martian surface, InSight's arm will scoop some additional soil on top to help it keep digging so it can take Mars' temperature."},
 {'title': "NASA's Perseverance Rover Will Peer Beneath Mars' Surface ",
  'paragraph': "The agency's newest rover will use the first ground-penetrating radar instrument on the Martian surface to help search for signs of past microbial life. "},
 {'title': 'AI Is Hel

In [110]:
db = client.web_scrapping_challenge_db
collection = db.mars_title
# Insert dictionary into MongoDB as a document
collection.insert_many(title_list)

##### This is the JPL website to get the featured high resolution image

In [111]:
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
url_base = "https://www.jpl.nasa.gov/"
browser.visit(url)
#https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg
time.sleep(1)
#images/wallpaper/PIA23341-1920x1200.jpg
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")
#style="background-image: url('/spaceimages/images/wallpaper/PIA23431-1920x1200.jpg')

avg_temps = soup.find("div", class_="carousel_container")

#a_all = avg_temps.find_all('a')[1].text
#td_all = avg_temps.find_all('td')[3].text
vol_str_data = 'dataset'
int_vol_str = ''

avg_vol_str_data = ''
avg_int_vol_str = ''
browser.find_by_css('li[bid=18663145091]')
mkt_vol_str_data = ''
mkt_int_vol_str = ''
img_list=[]
img_link = {}
paragraphs = avg_temps.find_all('div', class_="carousel_items")
for para in paragraphs:
    data_symbol={}
    try:
        avg_vol_str_data = para.find('article')
        data_symbol["img_url"] = avg_vol_str_data["style"]
        img_list.append(data_symbol)
    except:
        pass

mkt_vol_str_data=img_list[0]["img_url"]
mkt_vol_str_data=mkt_vol_str_data.replace('background-image: url(','')
mkt_vol_str_data=mkt_vol_str_data=mkt_vol_str_data.replace(');','')
mkt_vol_str_data =url_base+mkt_vol_str_data
mkt_vol_str_data= mkt_vol_str_data.replace("'",'')
img_link['img_link'] = mkt_vol_str_data
mkt_vol_str_data

'https://www.jpl.nasa.gov//spaceimages/images/wallpaper/PIA22911-1920x1200.jpg'

In [112]:
collection_link = db.mars_link_img
# Insert dictionary into MongoDB as a document
collection_link.insert_one(img_link)

#### This is to get the Mars facts, its attributes to store in local DB instance

In [113]:
url = "https://space-facts.com/mars/"
url_base = "https://www.jpl.nasa.gov/"
browser.visit(url)
#https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg
time.sleep(1)
#images/wallpaper/PIA23341-1920x1200.jpg
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")
#style="background-image: url('/spaceimages/images/wallpaper/PIA23431-1920x1200.jpg')

avg_temps = soup.find_all("div", class_="textwidget")


mars_data_list=[]
data_symbol={}
paragraphs = avg_temps[1].find_all('tr')
#for para in paragraphs:
#    data_symbol={}
#    try:
#        data_symbol["equatorial_diamegerr"] = para.find_all('td')[1].text
#        img_list.append(data_symbol)
#    except:
#        pass
#try :
data_symbol["equatorial_diameter"] = paragraphs[0].find_all('td')[1].text
data_symbol["polar_diameter"] = paragraphs[1].find_all('td')[1].text
data_symbol["mass"] = paragraphs[2].find_all('td')[1].text
data_symbol["moons"] = paragraphs[3].find_all('td')[1].text
data_symbol["orbit_distance"] = paragraphs[4].find_all('td')[1].text
data_symbol["orbit_period"] = paragraphs[5].find_all('td')[1].text
data_symbol["surface_temperature"] = paragraphs[6].find_all('td')[1].text
data_symbol["first_recorded"] = paragraphs[7].find_all('td')[1].text
data_symbol["recorded_by"] = paragraphs[8].find_all('td')[1].text
#except:
#    pass

#img_list.append(data_symbol)

#data_symbol['text']=paragraphs[0].find_all('tr').text

mars_data_list.append(data_symbol)
mars_data_list

[{'equatorial_diameter': '6,792 km',
  'polar_diameter': '6,752 km',
  'mass': '6.39 × 10^23 kg (0.11 Earths)',
  'moons': '2 (Phobos & Deimos)',
  'orbit_distance': '227,943,824 km (1.38 AU)',
  'orbit_period': '687 days (1.9 years)',
  'surface_temperature': '-87 to -5 °C',
  'first_recorded': '2nd millennium BC',
  'recorded_by': 'Egyptian astronomers'}]

In [114]:
collection_data = db.mars_data
# Insert dictionary into MongoDB as a document
collection_data.insert_many(mars_data_list)

##### This is for the Mars hemisphere data

In [115]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
url_base = "https://astrogeology.usgs.gov"
browser.visit(url)
#https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA23341-1920x1200.jpg
#https://astrogeology.usgs.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png
time.sleep(1)
#images/wallpaper/PIA23341-1920x1200.jpg
# Scrape page into Soup
html = browser.html
soup = bs(html, "html.parser")
#style="background-image: url('/spaceimages/images/wallpaper/PIA23431-1920x1200.jpg')

avg_temps = soup.find_all("div", class_="collapsible results")

#a_all = avg_temps.find_all('a')[1].text
#td_all = avg_temps.find_all('td')[3].text
vol_str_data = 'dataset'
int_vol_str = ''

avg_vol_str_data = ''
avg_int_vol_str = ''

mkt_vol_str_data = ''
mkt_int_vol_str = ''
hemisphere_img_list=[]
data_symbol={}
paragraphs = soup.find_all("div", class_="item")
for para in paragraphs:
    data_symbol={}
    try:
        data_symbol["img_src"] = url_base+para.find('img')["src"]
        data_symbol["title"] = para.find('h3').text
        data_symbol["description"] = para.find('p').text
        hemisphere_img_list.append(data_symbol)
    except:
        pass
#try :
#data_symbol["equatorial_diamegerr"] = paragraphs[0].find_all('td')[1].text
#data_symbol["polar_diameter"] = paragraphs[1].find_all('td')[1].text
#data_symbol["mass"] = paragraphs[2].find_all('td')[1].text
#data_symbol["moons"] = paragraphs[3].find_all('td')[1].text
#data_symbol["orbit_distance"] = paragraphs[4].find_all('td')[1].text
##data_symbol["orbit_period"] = paragraphs[5].find_all('td')[1].text
#data_symbol["surface_temperature"] = paragraphs[6].find_all('td')[1].text
#except:
#    pass

#img_list.append(data_symbol)

#data_symbol['text']=paragraphs[0].find_all('tr').text


hemisphere_img_list

[{'img_src': 'https://astrogeology.usgs.gov/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png',
  'title': 'Cerberus Hemisphere Enhanced',
  'description': 'Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…'},
 {'img_src': 'https://astrogeology.usgs.gov/cache/images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png',
  'title': 'Schiaparelli Hemisphere Enhanced',
  'description': 'Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…'},
 {'img_src': 'https://astrogeology.usgs.gov/cache/images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png',
  'title': 'Syrtis Major Hemisphere Enhanced',
  'description': 'Mosaic of the Syrtis Maj

In [116]:
collection_hemisphere = db.mars_hemisphere
# Insert dictionary into MongoDB as a document
collection_hemisphere.insert_many(hemisphere_img_list)